In [4]:
#general instructions

#2 or more patient encounters are required as the authors remove the last encounter while making predictions
# patient mortality is the outcome label and aquired from mimic. 
#2825 out of 7537 patients died (37.9%)
#Data set was divided into  train (5275/7537), validation (753/7537) and test (1509/7537)
# Once the optimal parameters were selected, the model was retrained by combining train and validation (6028/7537)
#
#
#
#

In [5]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time

In [6]:
# set seed - only really necessary to compare results with each other, but can remove from the submission
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = "~/Documents/DLH 598/mimic-iii-clinical-database-1.4"


# define data - separated to show which modules load (some take much longer than others)
#The commented out sections take a long time to load, and weren't immidiately useful at this stage
#Can uncommment when relevant

#TODO
# - reorganize into a definition() after  

In [7]:
Patients = pd.read_csv(DATA_PATH + '/PATIENTS.csv')

In [8]:
Diagnosis = pd.read_csv(DATA_PATH + '/DIAGNOSES_ICD.csv')

In [9]:
Procedures = pd.read_csv(DATA_PATH + '/D_ICD_PROCEDURES.csv')

In [10]:
#Prescriptions = pd.read_csv(DATA_PATH + '/PRESCRIPTIONS.csv')

In [11]:
#LabEvents = pd.read_csv(DATA_PATH + '/LABEVENTS.csv')

In [12]:
LabLabels = pd.read_csv(DATA_PATH + '/D_LABITEMS.csv')

In [13]:
Admissions = pd.read_csv(DATA_PATH + '/ADMISSIONS.csv')

In [14]:
ProcedureLabels = pd.read_csv(DATA_PATH + '/D_ITEMS.csv')

In [15]:
IcuStays = pd.read_csv(DATA_PATH + '/ICUSTAYS.csv')

In [16]:
#ChartEvents = pd.read_csv(DATA_PATH + '/CHARTEVENTS.csv')

In [17]:
# patient inclusion - Only ICU patients with 2 or more visits are included. Patient IDs gathered
#May be incorrect as the paper specifies only 7,537 patients in their list.

#Generate an ignore list for later ease of use.

unique, counts = np.unique(IcuStays[1:, 0], return_counts=True)

AllPatients = np.array(list(zip(unique, counts)))

PatientList = []
IgnoreList = []

for i in AllPatients:
    if i[1] >= 2:
        PatientList.append(i)
    else:
        IgnoreList.append(i[0])

print(len(PatientList))
print(len(IgnoreList))

InvalidIndexError: (slice(1, None, None), 0)

In [ ]:
# data statistics
white = 0
black = 0
hispanic = 0
asian = 0
other = 0


#Not yet working.... General idea.
for i in PatientList:
    count = 0
    if i == adm[:,1]:
        if adm[count,13] == "WHITE":
            white += 1
        if adm[count,13] == "BLACK":
            black += 1
        if adm[count,13] == "HISPANIC":
            hispanic += 1
        if adm[count,13] == "ASIAN":
            asian += 1
        else:
            other += 1
    count += 1
print(white)
print(black)

In [ ]:
#Age statistics


In [ ]:
#Sex statistics


In [ ]:
#ICD Code statistics


In [ ]:
#Medication statistics


In [ ]:
#Lab component statistics
#The paper mentioned significant removal of duplicate patient information at this step (20% of data removed)
#a total of 5,609,021 lab tests were removed as they were not used.


In [ ]:
#Outcome label = patient mortatility. Flag found in Admissions?


In [ ]:
#Data loader


In [ ]:
#Retain


In [ ]:
#Retain training


In [ ]:
#Clout model (The most important model to finish)


In [ ]:
#Clout training


In [ ]:
#Linear regression model


In [ ]:
#LR training


In [ ]:
#Evaluation (do all together? or can do separate?)


In [ ]:
#Summary conclusions


In [ ]:
#Summary Graphs etc...
